In [1]:
from coptpy import *

In [2]:
# Create environment
env = Envr()
# Create model 
model = env.createModel(name="OLS_simple")

Cardinal Optimizer v5.0.4. Build date Aug 19 2022
Copyright Cardinal Operations 2022. All Rights Reserved



Input five sets of sample data for solar cars

In [3]:
N, power, distance = multidict({
    '1': [10, 60],
    '2': [8, 55],
    '3': [13, 75],
    '4': [15, 81],
    '5': [9, 62]})

**Decision variables**:<br/>
**m**: the estimated coefficient in the linear relationship between power and distance;<br/>
**b**: the estimated constant in a linear relationship;<br/>
**OLS_positive_error($e_k^+$)**: positive estimation error for sample k, k∈N;<br/>
**OLS_negative_error($e_k^-$)**: negative estimation error for sample k, k∈N

In [4]:
#Add variables
OLS_positive_error = model.addVars(N)
OLS_negative_error = model.addVars(N)
m = model.addVar(name="m")
b = model.addVar(name="b")

**Constrains:**
$$e_k^+ - e_k^- = m \times power_k + b - distance_k,\quad k \in N$$
$$b, m, e_k^+, e_k^- \geq 0,\quad k \in N$$

In [5]:
#Add constraints
for k in N:
    model.addConstr(OLS_positive_error[k] - OLS_negative_error[k], COPT.EQUAL, power[k] * m + b - distance[k])

**Objective function**：$$\min \sum_{k=1}^N \lvert distance_k -m \times power_k + b \rvert$$ can be transformed into $$\min \sum_{k=1}^N (e_k^+ + e_k^-)$$

In [8]:
#Set Objective
model.setObjective(sum(OLS_positive_error[k]+OLS_negative_error[k] for k in N), sense=COPT.MINIMIZE)

In [9]:
#Solve the model
model.solve()

Model fingerprint: 183940f0

Hardware has 8 cores and 16 threads. Using instruction set X86_NATIVE (1)
Minimizing an LP problem

The original problem has:
    5 rows, 12 columns and 20 non-zero elements
The presolved problem has:
    5 rows, 12 columns and 20 non-zero elements

Starting the simplex solver using up to 8 threads

Method   Iteration           Objective  Primal.NInf   Dual.NInf        Time
Dual             0    0.0000000000e+00            5           0       0.01s
Dual             5    7.1443478745e+00            0           0       0.01s
Postsolving
Dual             5    7.1428571429e+00            0           0       0.01s

Solving finished
Status: Optimal  Objective: 7.1428571429e+00  Iterations: 5  Time: 0.01s


We have obtained the optimal solution

In [11]:
#Analyze solution
if model.status == COPT.OPTIMAL:
    # Optimal objective value
    print("optimal value：{:.4f}".format(model.objval))
    allvars = model.getVars()
    #Variable value
    print("\nValue of each variable:\n")
    for var in allvars:
        print("decision variable {0}, optimal solution is {1:.4f}".format(var.name, var.x))   

optimal value：7.1429

Value of each variable:

decision variable C(1), optimal solution is 2.4286
decision variable C(2), optimal solution is 0.0000
decision variable C(3), optimal solution is 0.0000
decision variable C(4), optimal solution is 0.0000
decision variable C(5), optimal solution is 0.0000
decision variable C(1), optimal solution is 0.0000
decision variable C(2), optimal solution is 0.0000
decision variable C(3), optimal solution is 1.4286
decision variable C(4), optimal solution is 0.0000
decision variable C(5), optimal solution is 3.2857
decision variable m, optimal solution is 3.7143
decision variable b, optimal solution is 25.2857


**Results**:<br/>
According to the results, the best-fit m and b are 3.7143 and 25.2587, respectively. The minimal sum of the absolute deviations between the observed distances and predicted distance is 7.1429.